In [1]:
!wget https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar -O tabular-dl-tabr.tar.gz
!tar -xvf tabular-dl-tabr.tar.gz

--2024-11-16 11:54:39--  https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar
Resolving huggingface.co (huggingface.co)... 13.35.210.66, 13.35.210.77, 13.35.210.61, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.66|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1b/25/1b25d6e2556c827abfaf6ba9da6021338691cafc39fe9d77986955ae0a69243d/0d74e4b96febb48fbe4dd2d851f8638b5738354f82c3183f92adbd2abf2f256b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.tar%3B+filename%3D%22data.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1732017279&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjAxNzI3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xYi8yNS8xYjI1ZDZlMjU1NmM4MjdhYmZhZjZiYTlkYTYwMjEzMzg2OTFjYWZjMzlmZTlkNzc5ODY5NTVhZTBhNjkyNDNkLzBkNzRlNGI5NmZlYmI0OGZiZTRkZDJkODUxZjg2MzhiNTczODM1NGY4MmMzMTgzZjkyYWRiZDJhYmYyZjI1NmI%

In [2]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 140.2 kB/s eta 0:00:00


In [4]:
!pip install psutil

In [3]:
import pandas as pd
import numpy as np
#import optuna
import torch.optim
import psutil

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import QuantileTransformer

In [ ]:
clf = TabNetClassifier()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [5]:
X_train = np.load('/content/data/classif-num-medium-0-credit/X_num_train.npy')
y_train = np.load('/content/data/classif-num-medium-0-credit/Y_train.npy')

X_valid = np.load('/content/data/classif-num-medium-0-credit/X_num_val.npy')
y_valid = np.load('/content/data/classif-num-medium-0-credit/Y_val.npy')

X_test = np.load('/content/data/classif-num-medium-0-credit/X_num_test.npy')
y_test = np.load('/content/data/classif-num-medium-0-credit/Y_test.npy')

In [6]:
normalizer = QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X_train.shape[0] // 30, 1000), 10),
            subsample=1_000_000_000,
            )
normalizer.fit_transform(X_train)

array([[-0.9945246 ,  0.3991821 , -5.1993375 , ...,  0.14394721,
        -5.1993375 , -5.1993375 ],
       [ 0.70878035, -0.7726002 ,  1.2188752 , ..., -5.1993375 ,
        -5.1993375 ,  0.8513439 ],
       [-5.1993375 , -1.9015584 ,  1.2188752 , ..., -5.1993375 ,
         1.2936593 , -5.1993375 ],
       ...,
       [ 1.5529194 ,  0.6509788 ,  1.2188752 , ..., -5.1993375 ,
        -5.1993375 ,  0.8513439 ],
       [-0.1352623 , -0.7726002 , -5.1993375 , ...,  0.14394721,
        -5.1993375 ,  1.4165093 ],
       [ 0.79665256, -0.91261363, -5.1993375 , ..., -5.1993375 ,
         1.2936593 , -5.1993375 ]], dtype=float32)

In [ ]:
#Baseline
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
)

epoch 0  | loss: 0.68516 | train_auc: 0.67334 | valid_auc: 0.67723 |  0:00:02s
epoch 1  | loss: 0.63397 | train_auc: 0.68989 | valid_auc: 0.70215 |  0:00:03s
epoch 2  | loss: 0.59455 | train_auc: 0.75285 | valid_auc: 0.75263 |  0:00:04s
epoch 3  | loss: 0.58448 | train_auc: 0.70174 | valid_auc: 0.70116 |  0:00:04s
epoch 4  | loss: 0.57622 | train_auc: 0.7245  | valid_auc: 0.72366 |  0:00:05s
epoch 5  | loss: 0.56399 | train_auc: 0.75419 | valid_auc: 0.75544 |  0:00:06s
epoch 6  | loss: 0.5503  | train_auc: 0.78582 | valid_auc: 0.78152 |  0:00:06s
epoch 7  | loss: 0.54258 | train_auc: 0.79779 | valid_auc: 0.79636 |  0:00:07s
epoch 8  | loss: 0.53529 | train_auc: 0.79706 | valid_auc: 0.79671 |  0:00:08s
epoch 9  | loss: 0.54337 | train_auc: 0.80292 | valid_auc: 0.80035 |  0:00:09s
epoch 10 | loss: 0.53636 | train_auc: 0.80618 | valid_auc: 0.80195 |  0:00:09s
epoch 11 | loss: 0.53925 | train_auc: 0.81335 | valid_auc: 0.814   |  0:00:10s
epoch 12 | loss: 0.5347  | train_auc: 0.81269 | vali

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
def objective(trial):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'momentum': trial.suggest_float('momentum', 0.01, 0.4),
    }

    model = TabNetClassifier(**params, n_a=params['n_d'])
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=['accuracy'], patience=10)

    # Evaluate model performance
    score = model.best_cost  # or any other metric

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2024-07-06 04:32:17,204] A new study created in memory with name: no-name-6dfff638-78d5-455a-ab98-0e7364494ba2
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.83746 | val_0_accuracy: 0.58143 |  0:00:00s
epoch 1  | loss: 0.66975 | val_0_accuracy: 0.57746 |  0:00:01s
epoch 2  | loss: 0.5863  | val_0_accuracy: 0.56455 |  0:00:01s
epoch 3  | loss: 0.5507  | val_0_accuracy: 0.51887 |  0:00:02s
epoch 4  | loss: 0.53365 | val_0_accuracy: 0.53078 |  0:00:03s
epoch 5  | loss: 0.53709 | val_0_accuracy: 0.61122 |  0:00:03s
epoch 6  | loss: 0.52418 | val_0_accuracy: 0.52036 |  0:00:04s
epoch 7  | loss: 0.51897 | val_0_accuracy: 0.72195 |  0:00:05s
epoch 8  | loss: 0.51724 | val_0_accuracy: 0.67627 |  0:00:05s
epoch 9  | loss: 0.52488 | val_0_accuracy: 0.69464 |  0:00:06s
epoch 10 | loss: 0.52344 | val_0_accuracy: 0.7001  |  0:00:06s
epoch 11 | loss: 0.51334 | val_0_accuracy: 0.71946 |  0:00:07s
epoch 12 | loss: 0.52216 | val_0_accuracy: 0.71599 |  0:00:08s
epoch 13 | loss: 0.51268 | val_0_accuracy: 0.71698 |  0:00:08s
epoch 14 | loss: 0.5109  | val_0_accuracy: 0.70159 |  0:00:09s
epoch 15 | loss: 0.50618 | val_0_accuracy: 0.73883 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 04:32:45,522] Trial 0 finished with value: 0.7562065541211519 and parameters: {'n_d': 57, 'n_steps': 3, 'gamma': 1.0020217428383593, 'n_independent': 5, 'n_shared': 2, 'momentum': 0.3327230063100925}. Best is trial 0 with value: 0.7562065541211519.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.58171 | val_0_accuracy: 0.52284 |  0:00:01s
epoch 1  | loss: 1.05012 | val_0_accuracy: 0.50497 |  0:00:02s
epoch 2  | loss: 1.04654 | val_0_accuracy: 0.57001 |  0:00:03s
epoch 3  | loss: 0.86941 | val_0_accuracy: 0.54767 |  0:00:05s
epoch 4  | loss: 0.79773 | val_0_accuracy: 0.57547 |  0:00:06s
epoch 5  | loss: 0.63798 | val_0_accuracy: 0.63108 |  0:00:07s
epoch 6  | loss: 0.6457  | val_0_accuracy: 0.66038 |  0:00:08s
epoch 7  | loss: 0.58855 | val_0_accuracy: 0.68669 |  0:00:10s
epoch 8  | loss: 0.58115 | val_0_accuracy: 0.66882 |  0:00:11s
epoch 9  | loss: 0.57842 | val_0_accuracy: 0.74181 |  0:00:12s
epoch 10 | loss: 0.56054 | val_0_accuracy: 0.74081 |  0:00:13s
epoch 11 | loss: 0.5601  | val_0_accuracy: 0.71946 |  0:00:15s
epoch 12 | loss: 0.55715 | val_0_accuracy: 0.70357 |  0:00:16s
epoch 13 | loss: 0.55865 | val_0_accuracy: 0.70209 |  0:00:17s
epoch 14 | loss: 0.54851 | val_0_accuracy: 0.73883 |  0:00:18s
epoch 15 | loss: 0.54197 | val_0_accuracy: 0.7145  |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 04:33:11,358] Trial 1 finished with value: 0.7418073485600795 and parameters: {'n_d': 38, 'n_steps': 8, 'gamma': 1.1675185940529527, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.2505827230371024}. Best is trial 0 with value: 0.7562065541211519.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 1.13804 | val_0_accuracy: 0.52483 |  0:00:01s
epoch 1  | loss: 0.96101 | val_0_accuracy: 0.52979 |  0:00:02s
epoch 2  | loss: 0.77571 | val_0_accuracy: 0.52085 |  0:00:04s
epoch 3  | loss: 0.74519 | val_0_accuracy: 0.54767 |  0:00:05s
epoch 4  | loss: 0.69298 | val_0_accuracy: 0.56802 |  0:00:06s
epoch 5  | loss: 0.65794 | val_0_accuracy: 0.59434 |  0:00:08s
epoch 6  | loss: 0.62211 | val_0_accuracy: 0.61817 |  0:00:09s
epoch 7  | loss: 0.61868 | val_0_accuracy: 0.60924 |  0:00:11s
epoch 8  | loss: 0.59787 | val_0_accuracy: 0.62264 |  0:00:12s
epoch 9  | loss: 0.6057  | val_0_accuracy: 0.62214 |  0:00:13s
epoch 10 | loss: 0.59216 | val_0_accuracy: 0.62066 |  0:00:15s
epoch 11 | loss: 0.63482 | val_0_accuracy: 0.63505 |  0:00:16s
epoch 12 | loss: 0.57839 | val_0_accuracy: 0.67925 |  0:00:18s
epoch 13 | loss: 0.58635 | val_0_accuracy: 0.67031 |  0:00:19s
epoch 14 | loss: 0.57135 | val_0_accuracy: 0.67974 |  0:00:20s
epoch 15 | loss: 0.56421 | val_0_accuracy: 0.6857  |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 05:13:19,687] Trial 98 finished with value: 0.7517378351539226 and parameters: {'n_d': 44, 'n_steps': 9, 'gamma': 1.258743858343868, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.03587462427858347}. Best is trial 64 with value: 0.7651439920556107.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.80275 | val_0_accuracy: 0.52979 |  0:00:00s
epoch 1  | loss: 0.6383  | val_0_accuracy: 0.55958 |  0:00:00s
epoch 2  | loss: 0.58375 | val_0_accuracy: 0.60079 |  0:00:01s
epoch 3  | loss: 0.57097 | val_0_accuracy: 0.69166 |  0:00:01s
epoch 4  | loss: 0.55265 | val_0_accuracy: 0.68669 |  0:00:01s
epoch 5  | loss: 0.54345 | val_0_accuracy: 0.65889 |  0:00:02s
epoch 6  | loss: 0.53567 | val_0_accuracy: 0.714   |  0:00:02s
epoch 7  | loss: 0.54122 | val_0_accuracy: 0.71251 |  0:00:03s
epoch 8  | loss: 0.53302 | val_0_accuracy: 0.71251 |  0:00:03s
epoch 9  | loss: 0.53468 | val_0_accuracy: 0.66038 |  0:00:03s
epoch 10 | loss: 0.53529 | val_0_accuracy: 0.65591 |  0:00:04s
epoch 11 | loss: 0.52101 | val_0_accuracy: 0.70953 |  0:00:04s
epoch 12 | loss: 0.52648 | val_0_accuracy: 0.74727 |  0:00:05s
epoch 13 | loss: 0.51431 | val_0_accuracy: 0.73635 |  0:00:05s
epoch 14 | loss: 0.51529 | val_0_accuracy: 0.73635 |  0:00:05s
epoch 15 | loss: 0.52093 | val_0_accuracy: 0.73337 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 05:13:39,209] Trial 99 finished with value: 0.7581926514399205 and parameters: {'n_d': 46, 'n_steps': 3, 'gamma': 1.5825696753093588, 'n_independent': 2, 'n_shared': 1, 'momentum': 0.010508402685219853}. Best is trial 64 with value: 0.7651439920556107.


epoch 49 | loss: 0.50216 | val_0_accuracy: 0.75074 |  0:00:19s

Early stopping occurred at epoch 49 with best_epoch = 39 and best_val_0_accuracy = 0.75819


In [ ]:
best_params

{'n_d': 53,
 'n_steps': 4,
 'gamma': 1.1083832817801176,
 'n_independent': 3,
 'n_shared': 2,
 'momentum': 0.02953222032864832}

In [7]:
best_params = {
    'n_d': 53,
    'n_steps': 4,
    'gamma': 1.1083832817801176,
    'n_independent': 3,
    'n_shared': 2,
    'momentum': 0.02953222032864832
    }

In [8]:
from pytorch_tabnet.callbacks import Callback

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"System RAM Usage: {mem_info.rss / (1024 ** 2):.2f} MB")
    if torch.cuda.is_available():
      print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")
      print(f"Cached GPU memory: {torch.cuda.memory_reserved() / (1024 ** 2):.2f} MB")

class MemoryUsageCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print_memory_usage()

In [9]:
# Train final model using best hyperparameters
best_model = TabNetClassifier(
    **best_params,
    n_a=best_params['n_d'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2)
)

best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    patience=16,
    callbacks=[MemoryUsageCallback()]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.95803 | train_accuracy: 0.523   | valid_accuracy: 0.51688 |  0:00:07s
System RAM Usage: 1178.63 MB
Allocated GPU memory: 23.63 MB
Cached GPU memory: 104.00 MB
epoch 1  | loss: 0.68105 | train_accuracy: 0.6003  | valid_accuracy: 0.61023 |  0:00:09s
System RAM Usage: 1179.04 MB
Allocated GPU memory: 23.63 MB
Cached GPU memory: 106.00 MB
epoch 2  | loss: 0.59175 | train_accuracy: 0.5101  | valid_accuracy: 0.52185 |  0:00:10s
System RAM Usage: 1179.04 MB
Allocated GPU memory: 23.63 MB
Cached GPU memory: 106.00 MB
epoch 3  | loss: 0.55795 | train_accuracy: 0.5008  | valid_accuracy: 0.52085 |  0:00:11s
System RAM Usage: 1179.04 MB
Allocated GPU memory: 23.63 MB
Cached GPU memory: 106.00 MB
epoch 4  | loss: 0.54908 | train_accuracy: 0.6826  | valid_accuracy: 0.67875 |  0:00:12s
System RAM Usage: 1179.04 MB
Allocated GPU memory: 23.63 MB
Cached GPU memory: 106.00 MB
epoch 5  | loss: 0.53835 | train_accuracy: 0.5937  | valid_accuracy: 0.5993  |  0:00:13s
System RAM Usage: 117

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.7249255213505462
Test AUC: 0.7893522177982696


In [ ]:
preds = best_model.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.71      0.74      0.72      2333
           1       0.73      0.71      0.72      2367

    accuracy                           0.72      4700
   macro avg       0.72      0.72      0.72      4700
weighted avg       0.72      0.72      0.72      4700



In [10]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.7249255213505462
Test AUC: 0.7893522177982696


In [11]:
fpr, tpr, thresholds = roc_curve(y_score=y_proba[:,1], y_true=y_test)

roc_data = pd.DataFrame({
    'fpr': fpr,
    'tpr': tpr
})

roc_data.to_csv('credit-TabNet.csv', index=False)